In [1]:
import xpress as xp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create a problem 
stage_1 = xp.problem(name='F_O stage 1')

Using the license file found in your Xpress installation. If you want to use this license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('C:/xpressmp/bin/xpauth.xpr')


In [3]:
# Import data into panda dataframes

df_rooms = pd.read_excel('Relevant Data Room.xlsx', header=0)
df_course_demand = pd.read_excel('Course Data.xlsx', header=0, sheet_name = 'Dict')[['Size']]
df_courses = pd.read_excel('Course Data.xlsx', header=0, sheet_name='Sheet1')
df_curricula = pd.read_excel('Course Data.xlsx', header=0, sheet_name='Programs')

df_rooms_types = pd.read_excel('Relevant Data Room.xlsx', header=0, sheet_name='ROOM_DIC')[['ROOM INDEX', 'ROOM NAME', 'CAP', 'ROOM LAYOUT']]

In [4]:
cu_1 = list(df_curricula['Y1'])
cu_2 = list(df_curricula['Y2'])
cu_3 = list(df_curricula['Y3'])

In [5]:
course_code = df_courses['Course Code'].to_numpy()

In [6]:
# Index sets
courses = list(df_courses['Course'])
course_code = df_courses['Course Code'].to_numpy()
demand = list(df_course_demand['Size'])
curricula = list(df_curricula['Code'])
timeslots = [i for i in range(42)] # Timeslots in a week, taking away Wednesday afternoon

days = ['M_O', 'T_O', 'W_O', 'TH_O', 'F_O'] #Days for odd week
rooms = list(df_rooms['ROOM INDEX'])
room_type = [1,2,3]
years = [1,2,3,4,5,'PGT']
#Lectures can happen in Theatre, Classroom
#Workshops happen in Teaching Studio, Classroom, Boardroom 
class_type = ['L', 'W']
workshop_pattern = [1, 2, 3]

capacities = sorted(np.append(df_rooms['CAP'].unique(), 0))
# Parameters 
lecture_hours = list(df_courses['Lecture - Duration'])
workshop_hours = list(df_courses['Workshop - Duration'])



# Create the variables
x = np.array([xp.var( name='x_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
                  for i in courses for j in timeslots for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(class_type))

y = np.array([xp.var( name='y_{0}_{1}_{2}_{3}'.format(i, j, s, k), vartype=xp.binary)
                  for i in courses for j in timeslots for s in capacities for k in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(capacities), len(class_type))

z = np.array([xp.var( name='z_{0}_{1}'.format(i, d), vartype=xp.binary)
                  for i in courses for d in days], dtype=xp.npvar).reshape(len(courses), len(days))

w = np.array([xp.var( name='w_{0}'.format(i))
                    for i in courses], dtype=xp.npvar)

r = np.array([xp.var( name='r_{0}_{1}_{2}'.format(cu, j, k), vartype=xp.binary)
                  for  cu in curricula for j in timeslots for k in range(3)], dtype=xp.npvar).reshape(len(curricula), len(timeslots), 3)

v = np.array([xp.var( name='v_{0}_{1}'.format(cu, j), vartype=xp.binary)
                  for cu in curricula for j in timeslots], dtype=xp.npvar).reshape(len(curricula), len(timeslots))

#counts number of clashes 
h = np.array([xp.var( name='h_{0}_{1}'.format(i, j), lb=0)
                    for i in years for j in timeslots], dtype=xp.npvar).reshape(len(years), len(timeslots))




In [7]:
# Add the variables to the problem
stage_1.addVariable(x, y, z, w, r, v, h)

In [8]:
capacities_2 = df_rooms['CAP']
room_layout = df_rooms_types['ROOM LAYOUT']
rooms_L = np.where(room_layout == 1)[0]
rooms_W = np.where(room_layout == 2)[0]
rooms_both = np.where(room_layout == 3)[0]

In [9]:
capacities_L = capacities_2[rooms_L]
capacities_W = capacities_2[rooms_W]
capacities_both = capacities_2[rooms_both]

In [10]:
#Add constraints

#C1: A course i in C should be assigned exactly L(c) lectures that is L(c)== #timeslots(c)
C1_L = [xp.Sum(x[i, j, 0] for j in (timeslots)) == lecture_hours[i] for i in range(len(courses))]
#for workshops
C1_W = [xp.Sum(x[i, j, 1] for j in (timeslots)) == workshop_hours[i] for i in range(len(courses))]

#C2: At a given time period we do not assign more courses than available rooms in general for both types

#!!! separate the types
C2 = [xp.Sum(x[i, j, k] for i in range(len(courses)) for k in range(len(class_type))) <= len(rooms) for j in (timeslots)]

#C3: Makes sure that if a room is assigned to a course in a time period, then course is assigned to that time period
C3 = [x[i,j,k] - y[i,j,s,k] >= 0 for i in range(len(courses)) for j in (timeslots) for k in range(len(class_type)) for s in range(len(capacities))]

#C4: ? Not needed, possible future implementation

#C5: Calculates if classtypes from a course are planned on a day 
C5 = [xp.Sum(x[i,j,k] for j in (timeslots)) - z[i,d] >= 0 for i in range(len(courses)) for k in (range(len(class_type))) for d in range(len((days)))]

#C6: Violation of mnd(c)
#C6 = Not needed, possible future implementation

#C7: Only one course from curriculum is planned at timeslot t
cu_1 = list(df_curricula['Y1'])
cu_2 = list(df_curricula['Y2'])
cu_3 = list(df_curricula['Y3'])


C7_1 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_1[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 0] == 0 for cu in range(len(curricula)) for j in (timeslots)]
C7_2 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_2[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 1] == 0 for cu in range(len(curricula)) for j in (timeslots)]
C7_3 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, np.array(cu_3[cu].split(', '))))[0] for k in range(len(class_type))) -r[cu, j, 2] == 0 for cu in range(len(curricula)) for j in (timeslots)]

#C8: Calculates curriculum compactness violation 
#C8 = [-r[cu,j-1] + r[cu,j] - r[cu, j+1] - v[cu,j] <= 0 for cu in range(len(curricula)) for j in [1,2,3,4,5,6,7]]

#C9: Demand is satisfied for classes

C9 = [y[i, j, s, k] == 0 for i in range(len(courses)) for j in timeslots for k in range(len(class_type)) for s in range(len(capacities)) if s < demand[i]]



df_courses_year = pd.read_excel('Course Data.xlsx', header=0, sheet_name='CoursesYears')
year1_courses = df_courses_year['Y1'].dropna().to_numpy()
year2_courses = df_courses_year['Y2'].dropna().to_numpy()
year3_courses = df_courses_year['Y3'].dropna().to_numpy()
year4_courses = df_courses_year['Y4'].dropna().to_numpy()
year5_courses = df_courses_year['Y5'].dropna().to_numpy()
pgt_courses = df_courses_year['PGT'].dropna().to_numpy()


#C10 
C10_1 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year1_courses))[0] for k in range(len(class_type))) - 1 <= h[0, j] for j in (timeslots)]
C10_2 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year2_courses))[0] for k in range(len(class_type)))- 1 <= h[1, j] for j in (timeslots)]
C10_3 = [xp.Sum(x[i,j,k]  for i in np.where(np.isin(course_code, year3_courses))[0] for k in range(len(class_type))) - 1 <= h[2, j] for j in (timeslots)]
C10_4 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year4_courses))[0] for k in range(len(class_type))) - 1 <= h[3, j] for j in (timeslots)]
C10_5 = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, year5_courses))[0] for k in range(len(class_type))) - 1 <= h[4, j] for j in (timeslots)]
C10_PGT = [xp.Sum(x[i,j,k] for i in np.where(np.isin(course_code, pgt_courses))[0] for k in range(len(class_type))) - 1 <= h[5, j] for j in (timeslots)]

 

C11_1 = [x[i, j, 1] <= 1 - x[i, j, 0] for i in range(len(courses)) for j in (timeslots)]
C11_2 = [x[i, j, 0] <= 1 - x[i, j, 1] for i in range(len(courses)) for j in (timeslots)]


C12_L = [xp.Sum(x[i, j, 0] * demand[i] for i in range(len(courses))) <= np.sum(capacities_L) + np.sum(capacities_both[:7]) for j in (timeslots)]
C12_W = [xp.Sum(x[i, j, 1] * demand[i] for i in range(len(courses))) <= np.sum(capacities_W) + np.sum(capacities_both[7:]) for j in (timeslots)]

In [11]:
#Add constraints 
stage_1.addConstraint(C1_L, C1_W, C2, C3, C5, C7_1, C7_2, C7_3, C10_1, C10_2, C10_3, C10_4, C10_5, C10_PGT, C11_1, C11_2, C12_L, C12_W)

In [12]:
#Write objective function 
stage_1.setObjective(xp.Sum(h[i,j] for i in range(len(years)) for j in range(len(timeslots))), sense = xp.minimize)

In [13]:
stage_1.solve()

FICO Xpress v9.2.2, Hyper, solve started 11:18:12, Apr 4, 2024
Heap usage: 91MB (peak 91MB, 3983KB system)
Minimizing MILP F_O stage 1 using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
    184566 rows       184830 cols       403586 elements    184531 entities
Presolved problem has:
      2192 rows         4116 cols        19572 elements      3864 entities
Presolve finished in 0 seconds
Heap usage: 114MB (peak 207MB, 3983KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  5.60e+02] / [ 1.37e-02,  1.09e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  3.99e+03] / [ 1.00e+00,  5.80e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 41, support set: 4116
 Number of orbits: 98, largest orbit: 42
 Row orbits: 50, row support: 2100
Will try to keep branch and b

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [14]:
results = np.array([])
timeslots = [i for i in range(9)] # Extract just monday
for j in timeslots:
    l_2 = np.array([stage_1.getSolution(x[i, j, k]) for i in range(len(course_code)) for k in range(len(class_type))])
    results = np.array([*results, l_2])


In [28]:
# Results for Monday, 9-10 am timeslot. This shows at even positions if a lecture from course i is planned at this timeslot.
# The elements at odd positions show if a workshop from course i happens at this timeslot

print(results[0])

[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0.  1.  1. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0.
 -0. -0.  1. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0.  1.  1. -0. -0. -0.
 -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0.
 -0.  1. -0. -0.]


In [15]:
# Create a problem 
stage_2 = xp.problem(name='F_O stage 2')

In [16]:
# Create the variables

u = np.array([xp.var( name='u_{0}_{1}_{2}_{3}'.format(i, j, k, l), vartype=xp.binary)
                  for i in courses for j in timeslots for k in rooms for l in class_type], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms), len(class_type))
#u = np.array([xp.var( name='u_{0}_{1}_{2}'.format(i, j, k), vartype=xp.binary)
#                  for i in courses for j in timeslots for k in rooms], dtype=xp.npvar).reshape(len(courses), len(timeslots), len(rooms))

t = np.array([xp.var( name='t_{0}_{1}'.format(i, j), vartype=xp.binary)
                  for i in courses for j in rooms], dtype=xp.npvar).reshape(len(courses), len(rooms))


In [17]:
# Add the variables to the problem
stage_2.addVariable(u, t)

In [18]:
# Constraints

b_2 = [xp.Sum(u[i, j, k, l] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms)) for l in range(len(class_type))]
c_2_L = [xp.Sum(u[i, j, k, 0] for k in np.hstack((rooms_L, rooms_both[:7]))) == 1 for j in timeslots for i in range(len(courses)) if results[j][2*i] == 1]
c_2_W = [xp.Sum(u[i, j, k, 1] for k in np.hstack((rooms_W, rooms_both[7:]))) == 1 for j in timeslots for i in range(len(courses)) if results[j][2*i + 1] == 1]
# d_2 = [xp.Sum(u[i, j, k, l] for j in timeslots for i in range(len(courses)) for l in range(len(class_type))) <= 1 for k in range(len(rooms))]
#b_2 = [xp.Sum(u[i, j, k] for j in timeslots) - len(timeslots) * t[i, k] <= 0 for i in range(len(courses)) for k in range(len(rooms))]
#c_2 = [xp.Sum(u[i, j, k] for k in range(len(rooms))) == 1 for j in timeslots for i in range(len(courses)) if np.any(l[j][2*i: 2*i+2] == 1)]
#d_2 = [xp.Sum(u[i, j, k] for i in range(len(courses)) for j in timeslots) <= 1 for k in range(len(rooms))]


In [19]:
#stage_2.addConstraint(b_2, c_2, d_2)
stage_2.addConstraint(b_2, c_2_L, c_2_W)

In [20]:
stage_2.setObjective(xp.Sum(t[i, j] for i in range(len(courses)) for j in range(len(rooms))), sense = xp.minimize)
stage_2.solve()

FICO Xpress v9.2.2, Hyper, solve started 11:18:32, Apr 4, 2024
Heap usage: 15MB (peak 15MB, 4830KB system)
Minimizing MILP F_O stage 2 using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
      5519 rows        51794 cols        56508 elements     51794 entities
Presolved problem has:
       344 rows         1436 cols         2564 elements      1436 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 21MB (peak 39MB, 4830KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  9.00e+00] / [ 1.25e-01,  1.50e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
Autoscaling applied standard scaling

Symmetric problem: generators: 305, support set: 1436
 Number of orbits: 12, largest orbit: 288
 Row orbits: 12, row support: 344
Will t

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [23]:
final_results = np.array([])
timeslots = [i for i in range(9)] 
for j in timeslots:
    for k in range(len(rooms)):
        l_2 = np.array([stage_2.getSolution(u[i, j, k, l]) for i in range(len(course_code)) for l in range(len(class_type))])
        final_results = np.array([*final_results, l_2])
